In [ ]:
# valutazione performance dei modelli nel calcolo dell' iC16

In [ ]:
import funzioni
from funzioni import *

import settings
from settings import *

In [ ]:
#==================================================================================================================================================================
#                                                                      OPERAZIONI SUL DB
#==================================================================================================================================================================

In [ ]:
%run preprocessing

In [ ]:
#==================================================================================================================================================================
#                                                            CALCOLO ARRAY CFU   
#==================================================================================================================================================================

In [ ]:
cfu = calcolaArrayCFU(df)

In [ ]:
#==================================================================================================================================================================
#                                                            SISTEMO DATI TEMPORALI
#==================================================================================================================================================================

In [ ]:
sistemaDatiTemporali(df,MESE_RIFERIMENTO)

In [ ]:
#==================================================================================================================================================================
#                                                            CREAZIONE MATRICOLE
#==================================================================================================================================================================

In [ ]:
#   tolgo colonne inutili
df.drop('DATA_SUP',axis=1,inplace=True)
df.drop('DATA_FREQ',axis=1,inplace=True)
df.drop('DATA_PRIMO_APPELLO',axis=1,inplace=True)
df.drop('AA_OFF_ID',axis=1,inplace=True)
df.drop('PESO',axis=1,inplace=True)
df.drop('DATA_CHIUSURA',axis=1,inplace=True)

In [ ]:
matricole = calcolaMatricole(df)

In [ ]:
#lo utilizzero' per prevedere le iscrizioni al secondo anno
matricoleConCodice=[]
for elem in matricole:
    copia = elem.copy(deep=True)
    matricoleConCodice.append(copia)


In [ ]:
# ELIMINO LE COLONNE INUTILI E CREO DB PER VISUALIZZARE GLI STUDENTI
for elem in matricole:
    elem.drop('STU_ID',axis=1,inplace=True)
    elem.drop('ULTIMO_TENTATIVO',axis=1,inplace=True)
    elem.drop('PRIMO_ANNO',axis=1,inplace=True)

#creo dataframe db per la visualizzazione più comoda
db = []
for elem in matricole:
    temp = elem.copy(deep=True)
    db.append(temp)

for elem in matricole:
    elem.drop('DES',axis=1,inplace=True)

In [ ]:
# VISUALIZZO STRUTTURA DATI
es = 0
print('matricole:')
display(matricole[es])    #   stato attuale
print('db:')
display(db[es])
# print('cfu: ')
# print(cfu[es])

In [ ]:
#esporto matricole
if ESPORTA_DATI:
    for i in range(len(matricole)):
        nome = 'export/X/' + str(i) + '.csv'
        matricole[i].to_csv(nome,index=False)

In [ ]:
#esporto cfu
if ESPORTA_DATI:
    with open('export/y.txt', 'w') as f:
        for elem in cfu:
            f.write(str(elem)+'\n')

In [ ]:
#==================================================================================================================================================================



#                                                   CREO X, Y CFU e Y ISCRIZIONE



#==================================================================================================================================================================

In [ ]:
X = matricole.copy()
#y per modelli di regressione
yregr = cfu.copy()

print('elementi X: ', len(X))
print('elementi y: ', len(yregr))

In [ ]:
#RIDUCO LE POSSIBILI CLASSI DI Y

print('CFU ottenuti: ')
print(yregr)

y=[]

for i in range(len(yregr)):
    if yregr[i] < CFU_RICHIESTI:
        y.append(0)
    else:
        y.append(1)

print('\ny risultante:')
print(y)


In [ ]:
#   CONVERSIONE A NUMPY ARRAY X e y

#conversione y
y = np.array(y)
yregr = np.array(yregr)

#conversione X
temp = [np.array(X[0].to_numpy().flatten())]
for i in range(1,len(X)):
    temp = np.append(temp,[X[i].to_numpy().flatten()],axis=0)
X = temp

In [ ]:
y_cfu = y.copy()

In [ ]:
#calcolo Y iscrizione

In [ ]:
#CARICO I DATI

dfOriginale = pd.read_excel('data/Informatica.xlsx')
dfOriginale = dfOriginale[colonneUtilizzate]

In [ ]:
matricoleConCodice[0]

In [ ]:
#vado a guardare classificare gli studenti tra attivi al II anno (se provano almeno un esame in tutto l' anno), oppure in inattivi

attiviSecondoAnno = []

#indica il numero di studenti iscritti all' anno successivo nel 2016,17,18
# (lo uso solo per fare analisi sui dati)
attiviNegliAnni = [0,0,0]

for i in range(len(matricoleConCodice)):

    #trovo id studente
    idStudente = matricoleConCodice[i].iloc[0,0]
    primoAnno = matricoleConCodice[i].iloc[0,4]

    attivoSecondoAnno = False
    #cerco un esame con l' id dello studente che lo studente ha provato o superato al secondo anno
    for j in range(len(dfOriginale)):
        if dfOriginale.loc[j,'STU_ID'] == idStudente:

            annoMaxAppello = dfOriginale.loc[j,'DATA_MAX_APPELLO']
            annoMaxAppello = str(annoMaxAppello)[0:4]
            
            annoSup = dfOriginale.loc[j,'DATA_SUP']
            annoSup = str(annoSup)[0:4]
            
            annoPrimoApp = dfOriginale.loc[j,'DATA_PRIMO_APPELLO']
            annoPrimoApp = str(annoPrimoApp)[0:4]
            
            if annoMaxAppello == str(primoAnno+1) or annoSup == str(primoAnno+1) or annoPrimoApp == str(primoAnno+1):
                attivoSecondoAnno=True
    
    if attivoSecondoAnno:
        attiviSecondoAnno.append(1)
        if primoAnno == 2016: attiviNegliAnni[0]+=1
        elif primoAnno == 2017: attiviNegliAnni[1]+=1
        elif primoAnno == 2018: attiviNegliAnni[2]+=1
    else:
        attiviSecondoAnno.append(0)
    
print('studenti attivi nell\' anno successivo nel:')
print('2016: ',attiviNegliAnni[0])
print('2017: ',attiviNegliAnni[1])
print('2018: ',attiviNegliAnni[2])


In [ ]:
print(attiviSecondoAnno)

In [ ]:
# calcolo il nuovo y
y_att = attiviSecondoAnno.copy()

In [ ]:
#   CONVERSIONE A NUMPY ARRAY X e y

#conversione y
y_att = np.array(y_att)

In [ ]:
#==================================================================================================================================================================



#                                                   SUDDIVISIONE IN TRAIN E TEST SET



#==================================================================================================================================================================

In [ ]:
len(y_cfu)
len(y_att)

In [ ]:
X_cfu_train, X_test, y_cfu_train, y_cfu_test = train_test_split(X,y_cfu, train_size=0.75, test_size=0.25,random_state=5)
X_att_train, X_test, y_att_train, y_att_test = train_test_split(X,y_att, train_size=0.75, test_size=0.25,random_state=5)
#   DIMENSIONI X, y
print('dimensioni X: ',X.shape)
print('dimensioni y: ',y.shape)

print('dimensioni X train 1: ',X_cfu_train.shape)
print('dimensioni X train 2: ',X_att_train.shape)

print('dimensioni y train 1: ',y_cfu_train.shape)
print('dimensioni y train 2: ',y_att_train.shape)

print('dimensioni X test: ',X_test.shape)

print('dimensioni y test1: ',y_cfu_test.shape)
print('dimensioni y test2: ',y_att_test.shape)

In [ ]:
#==================================================================================================================================================================



#                                                   BILANCIAMENTO TRAIN SET CFU



#==================================================================================================================================================================

In [ ]:
somma1 = 0
somma0 = 0
for elem in y_cfu_train:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y_cfu_train)*100,2)
perc1 = round((100 - perc0),2)
print('studenti con < 40 cfu: ',somma0)
print('studenti con >= 40 cfu: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'CFU < 40\n'+str(somma0)+'('+str(perc0)+'%)', 'CFU >= 40\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
# ax1.pie(sizes,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/sbilanciamentoDataset1.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
# oversampling

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X_cfu_train,y_cfu_train)
X_cfu_train = X_sm
y_cfu_train = y_sm

In [ ]:
somma1 = 0
somma0 = 0
for elem in y_cfu_train:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y_cfu_train)*100,2)
perc1 = round((100 - perc0),2)
print('studenti con < 40 cfu: ',somma0)
print('studenti con >= 40 cfu: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'CFU < 40\n'+str(somma0)+'('+str(perc0)+'%)', 'CFU >= 40\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
# ax1.pie(sizes,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/sbilanciamentoDataset1.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
#==================================================================================================================================================================



#                                                   BILANCIAMENTO TRAIN SET ATTIVITÀ



#==================================================================================================================================================================

In [ ]:
somma1 = 0
somma0 = 0
for elem in y_att_train:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y_att_train)*100,2)
perc1 = round((100 - perc0),2)
print('studenti non iscritti II anno',somma0)
print('studenti iscritti II anno: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Iscritti II anno\n'+str(somma0)+'('+str(perc0)+'%)', 'Non iscritti II anno\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.pie(sizes,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/sbilanciamentoDataset2.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
# oversampling

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X_att_train,y_att_train)
X_att_train = X_sm
y_att_train = y_sm

In [ ]:
somma1 = 0
somma0 = 0
for elem in y_att_train:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y_att_train)*100,2)
perc1 = round((100 - perc0),2)
print('studenti non iscritti II anno',somma0)
print('studenti iscritti II anno: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Iscritti II anno\n'+str(somma0)+'('+str(perc0)+'%)', 'Non iscritti II anno\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.pie(sizes,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/sbilanciamentoDataset2.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
#====================================

In [ ]:
#   DIMENSIONI X, y
print('dimensioni X: ',X.shape)
print('dimensioni y: ',y.shape)

print('dimensioni X train 1: ',X_cfu_train.shape)
print('dimensioni X train 2: ',X_att_train.shape)

print('dimensioni y train 1: ',y_cfu_train.shape)
print('dimensioni y train 2: ',y_att_train.shape)

print('dimensioni X test: ',X_test.shape)

print('dimensioni y test1: ',y_cfu_test.shape)
print('dimensioni y test2: ',y_att_test.shape)

In [ ]:
#==================================================================================================================================================================
#                                                             addestramento modelli
#==================================================================================================================================================================

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

model1 = make_pipeline(
        StackingEstimator(estimator=BernoulliNB(alpha=10.0, fit_prior=True)),
        GradientBoostingClassifier(learning_rate=0.1, max_depth=2, max_features=0.1, min_samples_leaf=7, min_samples_split=12, n_estimators=100, subsample=0.5)
    )

# Fix random state for all the steps in exported pipeline
set_param_recursive(model1.steps, 'random_state', 50)

model1.fit(X_cfu_train,y_cfu_train)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFwe, f_classif
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator

model2 = make_pipeline(
    SelectFwe(score_func=f_classif, alpha=0.023),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=False, loss="squared_hinge", penalty="l1", tol=0.01)),
    GradientBoostingClassifier(learning_rate=0.5, max_depth=8, max_features=0.1, min_samples_leaf=3, min_samples_split=11, n_estimators=100, subsample=0.6500000000000001)
)

model2.fit(X_att_train,y_att_train)

In [ ]:
#==================================================================================================================================================================
#                                                             valutazione accuratezza previsioni
#==================================================================================================================================================================

In [ ]:
#previsione cfu
prev_cfu = model1.predict(X_test)

print(prev_cfu)

corrette = 0
for corretta , prev in zip(y_cfu_test,prev_cfu):
    if corretta == prev : corrette +=1
print('corrette: ',corrette,'su ',len(y_cfu_test))

In [ ]:
#previsione iscrizione
prev_att = model2.predict(X_test)

print(prev_att)

corrette = 0
for corretta , prev in zip(y_att_test,prev_att):
    if corretta == prev : corrette +=1
print('corrette: ',corrette,'su ',len(y_att_test))

In [ ]:
#calcolo ic16 effettivo

studentiIdonei = 0  #studenti che rispettano i requisiti per contribuire alla percentualle ic16

for i in range(len(y_cfu_test)):
    # se lo studente ha raggiunto i 40 cfu ed è iscritto al secondo anno lo conto
    if y_cfu_test[i]==1 and y_att_test[i]==1:
        studentiIdonei +=1

ic16 = studentiIdonei / len(y_cfu_test) *100

In [ ]:
#ic16 previsto
studentiIdonei = 0  #studenti che rispettano i requisiti per contribuire alla percentualle ic16

for i in range(len(prev_cfu)):
    # se lo studente ha raggiunto i 40 cfu ed è iscritto al secondo anno lo conto
    if prev_cfu[i]==1 and prev_att[i]==1 :
        studentiIdonei +=1

ic16Prev = studentiIdonei / len(prev_cfu) *100

In [ ]:
print('ic16 effettivo:',round(ic16,2))
print('ic16 previsto: ',round(ic16Prev,2))
print('errore sulla previsione: ',round(abs(ic16-ic16Prev),2))

In [ ]:
#============================================= 
#calcolo errore medio
#============================================= 

In [ ]:
ic16Medio = 0
ic16PrevistoMedio = 0
erroreMedioPrevisione = 0
nloop = 300
for i in range(nloop):
    print(nloop-i)
    #definizione train e test set
    X_cfu_train, X_test, y_cfu_train, y_cfu_test = train_test_split(X,y_cfu, train_size=0.75, test_size=0.25,random_state=i)
    X_att_train, X_test, y_att_train, y_att_test = train_test_split(X,y_att, train_size=0.75, test_size=0.25,random_state=i)

    # oversampling cfu
    from imblearn.over_sampling import SMOTE
    smote = SMOTE(sampling_strategy='minority')
    X_sm, y_sm = smote.fit_resample(X_cfu_train,y_cfu_train)
    X_cfu_train = X_sm
    y_cfu_train = y_sm

    # oversampling attività
    smote = SMOTE(sampling_strategy='minority')
    X_sm, y_sm = smote.fit_resample(X_att_train,y_att_train)
    X_att_train = X_sm
    y_att_train = y_sm

    #addestramento modelli
    model1.fit(X_cfu_train,y_cfu_train)
    model2.fit(X_att_train,y_att_train)
    
    #calcolo previsioni
    prev_cfu = model1.predict(X_test)
    prev_att = model2.predict(X_test)

    # print('prev cfu:')
    # print(prev_cfu)
    # print('prev att:')
    # print(prev_att,'\n')

    #calcolo ic16 effettivo
    studentiIdonei = 0  #studenti che rispettano i requisiti per contribuire alla percentualle ic16
    for j in range(len(y_cfu_test)):
        # se lo studente ha raggiunto i 40 cfu ed è iscritto al secondo anno lo conto
        if y_cfu_test[j]==1 and y_att_test[j]==1:
            studentiIdonei +=1
    ic16 = studentiIdonei / len(y_cfu_test) *100

    #ic16 previsto
    studentiIdonei = 0  #studenti che rispettano i requisiti per contribuire alla percentualle ic16
    for j in range(len(prev_cfu)):
        # se lo studente ha raggiunto i 40 cfu ed è iscritto al secondo anno lo conto
        if prev_cfu[j]==1 and prev_att[j]==1 :
            studentiIdonei +=1
    ic16Prev = studentiIdonei / len(prev_cfu) *100
    
    errore = abs(ic16 - ic16Prev)

    ic16Medio += ic16
    ic16PrevistoMedio += ic16Prev
    erroreMedioPrevisione += errore
    
    print('nuovo errore: ',errore)
    print('errore medio: ',erroreMedioPrevisione/(i+1),'\n')

ic16Medio = round(ic16Medio/nloop,2)
ic16PrevistoMedio = round(ic16PrevistoMedio/nloop,2)
erroreMedioPrevisione = round(erroreMedioPrevisione/nloop,2)

print('ic16 medio: ',ic16Medio)
print('ic16 medio previsto: ',ic16PrevistoMedio)
print('errore medio sulla previsione: ',erroreMedioPrevisione)

In [ ]:
MESE_RIFERIMENTO

In [ ]:
# errore = [
#     9.88,
#     4.59,
#     4.50,
#     4.62,
#     4.56,
#     4.07,
#     3.61,
#     3.64,
#     2.31,
#     2.29,
#     1.73,
#     1.71,
# ]